In [35]:
import numpy as np
import pandas as pd
from bert_score import score as bert_score
from nlgeval import NLGEval


In [ ]:
df = pd.read_csv('adaptive_speak_eval_all.csv')
df.head(2)

In [67]:
refs = df['golden utt'].tolist()
refs_for_nlgeval = [[ref] for ref in refs]
hyps = {
    "h0": df['original utt'].tolist(),
    "h0' 0": df['adapted utt s0'].tolist(),
    # "h0' 1": df['adapted utt s1'].tolist(),
    # "h0' 2": df['adapted utt s2'].tolist()
}


In [ ]:
nlgeval = NLGEval(no_skipthoughts=True, no_glove=True, metrics_to_omit=['METEOR'])

score_dicts = []
for hyp_type in hyps:
    metrics_dict = nlgeval.compute_metrics(refs_for_nlgeval, hyps[hyp_type])
    (P, R, F)= bert_score(
        hyps[hyp_type],
        refs,
        lang="en",
        return_hash=False,
        model_type="microsoft/deberta-xlarge-mnli",
    )
    metrics_dict['BERT_P'] = P.mean().item()
    metrics_dict['BERT_R'] = R.mean().item()
    metrics_dict['BERT_F'] = F.mean().item()
    score_dicts.append(metrics_dict)

score_df = pd.DataFrame(score_dicts, index=hyps.keys())

In [ ]:
score_df

In [ ]:
axes = score_df.plot.bar(rot=0, subplots=True, figsize=(10,20), legend=None)
# axes[1].legend()